In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!cp /content/drive/MyDrive/kuliah/Smester\ \5/big\ \data\ \lanjut/spark-3.0.0-bin-hadoop3.2.tgz .

In [ ]:
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!java -version

openjdk version "11.0.16" 2022-07-19
OpenJDK Runtime Environment (build 11.0.16+8-post-Ubuntu-0ubuntu118.04)
OpenJDK 64-Bit Server VM (build 11.0.16+8-post-Ubuntu-0ubuntu118.04, mixed mode, sharing)


In [ ]:
!pip install -q findspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 42 kB/s 
     |████████████████████████████████| 199 kB 53.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=7898c6e7494d3c1246453ed624396a026e894dba923b57021316f8dca42f25ee
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
print(spark)
print(spark.version)

3.3.1


In [ ]:
sample_list = [('Mona',20), ('Jennifer',34),('John',20), ('Jim',26)]

In [ ]:
rdd = sc.parallelize(sample_list)
names_df = spark.createDataFrame(rdd, schema=['Name', 'Age'])

print("The type of names_df is", type(names_df))

The type of names_df is <class 'pyspark.sql.dataframe.DataFrame'>


In [ ]:
!wget -P '/content/drive/MyDrive/kuliah/Smester 5/big data lanjut/p8' https://raw.githubusercontent.com/ariflaksito/bigdata_pyspark/master/people.csv

--2022-11-02 06:41:52--  https://raw.githubusercontent.com/ariflaksito/bigdata_pyspark/master/people.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4280052 (4.1M) [text/plain]
Saving to: ‘/content/drive/MyDrive/kuliah/Smester 5/big data lanjut/p8/people.csv.1’

people.csv.1        100%[===================>]   4.08M  --.-KB/s    in 0.1s    

2022-11-02 06:41:53 (33.6 MB/s) - ‘/content/drive/MyDrive/kuliah/Smester 5/big data lanjut/p8/people.csv.1’ saved [4280052/4280052]



In [ ]:
people_df = spark.read.csv('/content/drive/MyDrive/kuliah/Smester 5/big data lanjut/p8/people.csv', header=True, inferSchema=True)

print("The type of people_df is", type(people_df))

The type of people_df is <class 'pyspark.sql.dataframe.DataFrame'>


In [ ]:
people_df.show(10)

print("There are {} rows in the people_df DataFrame.".format(people_df.count()))

print("There are {} columns in the people_df DataFrame and their names are {}".format(len(people_df.columns), people_df.columns))

+---+---------+----------------+------+-------------------+
|_c0|person_id|            name|   sex|      date of birth|
+---+---------+----------------+------+-------------------+
|  0|      100|  Penelope Lewis|female|1990-08-31 00:00:00|
|  1|      101|   David Anthony|  male|1971-10-14 00:00:00|
|  2|      102|       Ida Shipp|female|1962-05-24 00:00:00|
|  3|      103|    Joanna Moore|female|2017-03-10 00:00:00|
|  4|      104|  Lisandra Ortiz|female|2020-08-05 00:00:00|
|  5|      105|   David Simmons|  male|1999-12-30 00:00:00|
|  6|      106|   Edward Hudson|  male|1983-05-09 00:00:00|
|  7|      107|    Albert Jones|  male|1990-09-13 00:00:00|
|  8|      108|Leonard Cavender|  male|1958-08-08 00:00:00|
|  9|      109|  Everett Vadala|  male|2005-05-24 00:00:00|
+---+---------+----------------+------+-------------------+
only showing top 10 rows

There are 100000 rows in the people_df DataFrame.
There are 5 columns in the people_df DataFrame and their names are ['_c0', 'person_i

In [ ]:
# select name , sex , date of birth from people_df and insert to people_df_sub variabel
people_df_sub = people_df.select('name', 'sex', 'date of birth')

people_df_sub.show(10)

# remove duplicate from people_df_sub and remove Duplicates
people_df_sub_nodup = people_df_sub.dropDuplicates()

+----------------+------+-------------------+
|            name|   sex|      date of birth|
+----------------+------+-------------------+
|  Penelope Lewis|female|1990-08-31 00:00:00|
|   David Anthony|  male|1971-10-14 00:00:00|
|       Ida Shipp|female|1962-05-24 00:00:00|
|    Joanna Moore|female|2017-03-10 00:00:00|
|  Lisandra Ortiz|female|2020-08-05 00:00:00|
|   David Simmons|  male|1999-12-30 00:00:00|
|   Edward Hudson|  male|1983-05-09 00:00:00|
|    Albert Jones|  male|1990-09-13 00:00:00|
|Leonard Cavender|  male|1958-08-08 00:00:00|
|  Everett Vadala|  male|2005-05-24 00:00:00|
+----------------+------+-------------------+
only showing top 10 rows



In [ ]:
resultduplicate = people_df_sub.groupBy('name','sex','date of birth').count().filter("count > 1")
#resultduplicate = resultduplicate.filter("count > 1")
resultduplicate.show()

+-------------+------+--------------------+-----+
|         name|   sex|       date of birth|count|
+-------------+------+--------------------+-----+
| Robert Smith|  male|+20175-02-28 00:0...|    2|
|Kathryn Davis|female|+20175-02-28 00:0...|    2|
+-------------+------+--------------------+-----+



In [ ]:
print("There are {} rows in the people_df DataFrame after remove duplicate.".format(people_df_sub_nodup.count()))


There are 99998 rows in the people_df DataFrame after remove duplicate.


In [ ]:
# data sesudah remove duplicate
print("There are {} rows in the people_df DataFrame after remove duplicate.".format(people_df_sub_nodup.count()))
print("There are {} columns int the people_df after remove duplicate and their name are {}".format(len(people_df_sub_nodup.columns), people_df_sub_nodup.columns))
#print("There are {} columns in the people_df DataFrame and their names are {}".format(len(people_df.columns), people_df.columns))


There are 99998 rows in the people_df DataFrame after remove duplicate.
There are 3 columns int the people_df after remove duplicate and their name are ['name', 'sex', 'date of birth']


In [ ]:
print('Jumlah baris sebelum remove duplicate = ', people_df.count())
print('Jumlah baris setelah remove duplicate = ', people_df_sub_nodup.count())

Jumlah baris sebelum remove duplicate =  100000
Jumlah baris setelah remove duplicate =  99998


In [ ]:
# jumlah data sebelum di remove duplicate 100000 
# jumlah data setelah di remove duplicate 99998

In [ ]:
print('Jumlah baris sebelum remove duplicate = ', people_df.count())
people_df_sub_nodup = people_df_sub.dropDuplicates()
print('Jumlah baris setelah remove duplicate = ', people_df_sub_nodup.count())
# melihat data yang duplicate 
resultduplicate = people_df_sub.groupBy('name','sex','date of birth').count().filter("count > 1")
resultduplicate.show()

Jumlah baris sebelum remove duplicate =  100000
Jumlah baris setelah remove duplicate =  99998
+-------------+------+--------------------+-----+
|         name|   sex|       date of birth|count|
+-------------+------+--------------------+-----+
| Robert Smith|  male|+20175-02-28 00:0...|    2|
|Kathryn Davis|female|+20175-02-28 00:0...|    2|
+-------------+------+--------------------+-----+

